# JsonOutputParser

- Author: [최재훈](https://github.com/ash-hun)
- Design: 
- Peer Review: 
- This is a part of [LangChain Open Tutorial](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial)

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/langchain-ai/langchain-academy/blob/main/module-4/sub-graph.ipynb) [![Open in LangChain Academy](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66e9eba12c7b7688aa3dbb5e_LCA-badge-green.svg)](https://academy.langchain.com/courses/take/intro-to-langgraph/lessons/58239937-lesson-2-sub-graphs)

## Overview

이 튜토리얼은 JsonOutputParser의 구현을 다룹니다. 

JsonOutputParser는 사용자가 원하는 JSON 스키마를 지정할 수 있게 해주는 도구입니다. 이 도구는 Large Language Model (LLM)이 데이터를 조회하고 결과를 도출할 때, 지정된 스키마에 맞게 JSON 형식으로 데이터를 반환할 수 있도록 설계되었습니다.

LLM이 데이터를 정확하고 효율적으로 처리하여 사용자가 원하는 형태의 JSON을 생성하기 위해서는, 모델의 용량(예: 인텔리전스)이 충분히 커야 합니다. 예를 들어, llama-70B 모델은 llama-8B 모델보다 더 큰 용량을 가지고 있어 보다 복잡한 데이터를 처리하는 데 유리합니다.

**[참고]**

`JSON (JavaScript Object Notation)` 은 데이터를 저장하고 구조적으로 전달하기 위해 사용되는 경량의 데이터 교환 포맷입니다. 웹 개발에서 매우 중요한 역할을 하며, 서버와 클라이언트 간의 통신을 위해 널리 사용됩니다. JSON은 읽기 쉽고, 기계가 파싱하고 생성하기 쉬운 텍스트를 기반으로 합니다.

JSON의 기본 구조
JSON 데이터는 이름(키)과 값의 쌍으로 이루어져 있습니다. 여기서 "이름"은 문자열이고, "값"은 다양한 데이터 유형일 수 있습니다. JSON은 두 가지 기본 구조를 가집니다:

- 객체: 중괄호 {}로 둘러싸인 키-값 쌍의 집합입니다. 각 키는 콜론 :을 사용하여 해당하는 값과 연결되며, 여러 키-값 쌍은 쉼표 ,로 구분됩니다.
- 배열: 대괄호 []로 둘러싸인 값의 순서 있는 목록입니다. 배열 내의 값은 쉼표 ,로 구분됩니다.

```json
{
  "name": "John Doe",
  "age": 30,
  "is_student": false,
  "skills": ["Java", "Python", "JavaScript"],
  "address": {
    "street": "123 Main St",
    "city": "Anytown"
  }
}
```

### Table of Contents

- [Overview](#overview)
- [Environement Setup](#environment-setup)
- [Pydantic과 함께 JsonOutputParser 사용](#create-a-basic-pdf-based-retrieval-chain)
- [Pydantic없이 JsonOutputParser 사용](#query-routing-and-document-evaluation)

### References

- [LangChain: Query Construction](https://blog.langchain.dev/query-construction/)
- [LangGraph: Self-Reflective RAG](https://blog.langchain.dev/agentic-rag-with-langgraph/)
- [Adaptive-RAG: Learning to Adapt Retrieval-Augmented Large Language Models through Question Complexity](https://arxiv.org/abs/2403.14403)
----

## Environment Setup

Set up the environment. You may refer to [Environment Setup](https://wikidocs.net/257836) for more details.

**[Note]**
- `langchain-opentutorial` is a package that provides a set of easy-to-use environment setup, useful functions and utilities for tutorials. 
- You can checkout the [`langchain-opentutorial`](https://github.com/LangChain-OpenTutorial/langchain-opentutorial-pypi) for more details.

In [1]:
%%capture --no-stderr
!pip install langchain-opentutorial

In [3]:
# Install required packages
from langchain_opentutorial import package

package.install(
    [
        "langsmith",
        "langchain",
        "langchain_core",
        "langchain-anthropic",
        "langchain_community",
        "langchain_text_splitters",
        "langchain_openai",
    ],
    verbose=False,
    upgrade=False,
)


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
# Set environment variables
from langchain_opentutorial import set_env

set_env(
    {
        "OPENAI_API_KEY": "",
        "LANGCHAIN_API_KEY": "",
        "LANGCHAIN_TRACING_V2": "true",
        "LANGCHAIN_ENDPOINT": "https://api.smith.langchain.com",
        "LANGCHAIN_PROJECT": "JsonOutputParser",  # title 과 동일하게 설정해 주세요
    }
)

Environment variables have been set successfully.


아래 내용도 반영!

In [ ]:
# 
from dotenv import load_dotenv

load_dotenv()

## Pydantic과 함께 JsonOutputParser 사용

여기서는 PDF 문서를 기반으로 Retrieval Chain 을 생성합니다. 가장 단순한 구조의 Retrieval Chain 입니다.

단, LangGraph 에서는 Retirever 와 Chain 을 따로 생성합니다. 그래야 각 노드별로 세부 처리를 할 수 있습니다.

**참고**
- 이전 튜토리얼에서 다룬 내용이므로, 자세한 설명은 생략합니다.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

In [ ]:
# OpenAI 객체를 생성합니다.
model = ChatOpenAI(temperature=0, model_name="gpt-4o")

원하는 출력 구조를 정의합니다.

In [ ]:
# 원하는 데이터 구조를 정의합니다.
class Topic(BaseModel):
    description: str = Field(description="주제에 대한 간결한 설명")
    hashtags: str = Field(description="해시태그 형식의 키워드(2개 이상)")

`JsonOutputParser`를 사용하여 파서를 설정하고, 프롬프트 템플릿에 지시사항을 주입합니다.

In [ ]:
# 질의 작성
question = "지구 온난화의 심각성 대해 알려주세요."

# 파서를 설정하고 프롬프트 템플릿에 지시사항을 주입합니다.
parser = JsonOutputParser(pydantic_object=Topic)
print(parser.get_format_instructions())

In [ ]:
# 프롬프트 템플릿을 설정합니다.
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 친절한 AI 어시스턴트 입니다. 질문에 간결하게 답변하세요."),
        ("user", "#Format: {format_instructions}\n\n#Question: {question}"),
    ]
)

prompt = prompt.partial(format_instructions=parser.get_format_instructions())

# 체인을 구성합니다.
chain = prompt | model | parser

# 체인을 호출하여 쿼리 실행
answer = chain.invoke({"question": question})

In [ ]:
# 타입을 확인합니다.
type(answer)

In [ ]:
# answer 객체를 출력합니다.
answer

## Pydantic을 사용하지 않고 JsonOutputParser 사용

Pydantic 없이도 이 기능을 사용할 수 있습니다. 이 경우 JSON을 반환하도록 요청하지만, 스키마가 어떻게 되어야 하는지에 대한 구체적인 정보는 제공하지 않습니다.

In [ ]:
# 질의 작성
question = "지구 온난화에 대해 알려주세요. 온난화에 대한 설명은 `description`에, 관련 키워드는 `hashtags`에 담아주세요."

# JSON 출력 파서 초기화
parser = JsonOutputParser()

# 프롬프트 템플릿을 설정합니다.
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 친절한 AI 어시스턴트 입니다. 질문에 간결하게 답변하세요."),
        ("user", "#Format: {format_instructions}\n\n#Question: {question}"),
    ]
)

# 지시사항을 프롬프트에 주입합니다.
prompt = prompt.partial(format_instructions=parser.get_format_instructions())

# 프롬프트, 모델, 파서를 연결하는 체인 생성
chain = prompt | model | parser

# 체인을 호출하여 쿼리 실행
response = chain.invoke({"question": question})

# 출력을 확인합니다.
print(response)